In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import numpy as np 

print(sklearn.__version__)
import pickle

import numpy as np
from pycrfsuite import Tagger
from flexcrf_tp.models.linear_chain import (_feat_fun_values,
                                            _compute_all_potentials,
                                            _forward_score,
                                            _backward_score,
                                            _partition_fun_value,
                                            _posterior_score)

from flexcrf_tp.crfsuite2flexcrf import convert_data_to_flexcrf

0.18.1


# Let's use CoNLL 2002 data to build a NER system

CoNLL2002 corpus is available in NLTK. We use Spanish data.

In [2]:
nltk.corpus.conll2002.fileids()

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [3]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

Wall time: 5.14 s


Data format:

In [4]:
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

## Features

Next, define some features. In this example we use word identity, word suffix, word shape and word POS tag; also, some information from nearby words is used. 

This makes a simple baseline, but you certainly can add and remove some features to get (much?) better results - experiment with it.

In [5]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 1:
        word1 = sent[i-1][0]
        word2 = sent[i-2][0]
        postag1 = sent[i-1][1]
        postag2 = sent[i-2][1]
        
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],

            '-2:word.lower=' + word2.lower(),
            '-2:word.istitle=%s' % word2.istitle(),
            '-2:word.isupper=%s' % word2.isupper(),
            '-2:postag=' + postag2,
            '-2:postag[:2]=' + postag2[:2],
        ])
        
    elif i < len(sent)-2:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        word2 = sent[i+2][0]
        postag2 = sent[i+2][1]
        
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
            
            '+2:word.lower=' + word2.lower(),
            '+2:word.istitle=%s' % word2.istitle(),
            '+2:word.isupper=%s' % word2.isupper(),
            '+2:postag=' + postag2,
            '+2:postag[:2]=' + postag2[:2],
        ])
        
    elif i > 0 and i < len(sent) - 1:
        word_a_1 = sent[i + 1][0]
        postag_a_1 = sent[i + 1][1]
        word_b_1 = sent[i - 1][0]
        postag_b_1 = sent[i - 1][1]
        
        features.extend([
            # word sequence
            '+1:word.word='  + word.lower() + word_a_1.lower(),
            '-1:word.word='  + word_b_1.lower() + word.lower(),

            'word.word.word=' + word_b_1.lower() + word.lower() + word_a_1.lower(), 

            # tag sequence    
            '+1:postag.postag='  + postag + postag_a_1,
            '-1:postag.postag=' + postag_b_1 + postag, 

            'postag.postag.postag=' + postag_b_1 + postag + postag_a_1
        ])
        
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]    

This is what word2features extracts:

In [6]:
sent2features(train_sents[0])[0]

['bias',
 'word.lower=melbourne',
 'word[-3:]=rne',
 'word[-2:]=ne',
 'word.isupper=False',
 'word.istitle=True',
 'word.isdigit=False',
 'postag=NP',
 'postag[:2]=NP',
 '+1:word.lower=(',
 '+1:word.istitle=False',
 '+1:word.isupper=False',
 '+1:postag=Fpa',
 '+1:postag[:2]=Fp',
 '+2:word.lower=australia',
 '+2:word.istitle=True',
 '+2:word.isupper=False',
 '+2:postag=NP',
 '+2:postag[:2]=NP']

Extract the features from the data:

In [7]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

Wall time: 4.54 s


## Train the model

To train the model, we create pycrfsuite.Trainer, load the training data and call 'train' method. 
First, create pycrfsuite.Trainer and load the training data to CRFsuite:

In [8]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

Wall time: 5.66 s


Set training parameters. We will use L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.

In [9]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

Possible parameters for the default training algorithm:

In [10]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

Train the model:

In [11]:
%%time
trainer.train('conll2002-esp.crfsuite')

Wall time: 27.6 s


trainer.train saves model to a file:

In [12]:
!ls -lh ./conll2002-esp.crfsuite

-rw-r--r-- 1 Mohamed 197609 538K mars  28 19:24 ./conll2002-esp.crfsuite


We can also get information about the final state of the model by looking at the trainer's logparser. If we had tagged our input data using the optional group argument in add, and had used the optional holdout argument during train, there would be information about the trainer's performance on the holdout set as well. 

In [13]:
trainer.logparser.last_iteration

{'active_features': 10202,
 'error_norm': 313.904088,
 'feature_norm': 80.526738,
 'linesearch_step': 1.0,
 'linesearch_trials': 1,
 'loss': 14205.264353,
 'num': 50,
 'scores': {},
 'time': 0.505}

We can also get this information for every step using trainer.logparser.iterations

In [14]:
print(len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

50 {'active_features': 10202, 'linesearch_trials': 1, 'feature_norm': 80.526738, 'linesearch_step': 1.0, 'error_norm': 313.904088, 'scores': {}, 'loss': 14205.264353, 'time': 0.505, 'num': 50}


## Make predictions

To use the trained model, create pycrfsuite.Tagger, open the model and use "tag" method:

In [15]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')

Let's tag a sentence to see how it works:

In [16]:
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)), end='\n\n')

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

La Coruña , 23 may ( EFECOM ) .

Predicted: B-LOC I-LOC O O O O B-ORG O O
Correct:   B-LOC I-LOC O O O O B-ORG O O


## Evaluate the model

In [17]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

Predict entity labels for all sentences in our testing set ('testb' Spanish data):

In [18]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

Wall time: 866 ms


..and check the result. Note this report is not comparable to results in CONLL2002 papers because here we check per-token results (not per-entity). Per-entity numbers will be worse.  

In [19]:
print(bio_classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      B-LOC       0.79      0.75      0.77      1084
      I-LOC       0.69      0.64      0.66       325
     B-MISC       0.71      0.50      0.58       339
     I-MISC       0.62      0.55      0.58       557
      B-ORG       0.81      0.82      0.81      1400
      I-ORG       0.83      0.81      0.82      1104
      B-PER       0.80      0.88      0.84       735
      I-PER       0.87      0.93      0.90       634

avg / total       0.79      0.77      0.78      6178



## Let's check what classifier learned

In [20]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
B-PER  -> I-PER   6.120515
B-LOC  -> I-LOC   4.725438
I-MISC -> I-MISC  4.715527
B-ORG  -> I-ORG   4.272482
I-ORG  -> I-ORG   4.252518
I-PER  -> I-PER   4.209032
I-LOC  -> I-LOC   4.185278
B-MISC -> I-MISC  3.732494
O      -> B-MISC  1.412869
O      -> O       1.223342
O      -> B-ORG   0.873367
O      -> B-LOC   0.771999
I-PER  -> B-LOC   0.765577
O      -> B-PER   0.401441
B-ORG  -> B-LOC   0.141326

Top unlikely transitions:
B-MISC -> I-LOC   -2.818435
I-ORG  -> I-LOC   -2.820726
B-LOC  -> I-ORG   -2.836072
B-ORG  -> B-ORG   -2.871635
I-LOC  -> I-ORG   -3.013622
I-PER  -> I-ORG   -3.015370
B-MISC -> B-MISC  -3.040931
B-PER  -> B-PER   -3.094318
I-MISC -> I-LOC   -3.120213
B-MISC -> I-ORG   -3.436320
I-MISC -> I-ORG   -3.809124
O      -> I-PER   -7.074274
O      -> I-MISC  -7.764248
O      -> I-LOC   -8.066906
O      -> I-ORG   -8.124627


We can see that, for example, it is very likely that the beginning of an organization name (B-ORG) will be followed by a token inside organization name (I-ORG), but transitions to I-ORG from tokens with other labels are penalized. Also note I-PER -> B-LOC transition: a positive weight means that model thinks that a person name is often followed by a location.

Check the state features:

In [21]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
9.476269 B-ORG  word.lower=efe-cantabria
8.512838 B-ORG  word.lower=psoe-progresistas
5.406334 O      word.lower=mar-23-may-2000
4.915326 I-PER  word.lower=gándara
4.856425 B-ORG  word.lower=xfera
4.716338 B-ORG  word.lower=telefónica
4.641311 I-MISC -1:word.lower=ibex
4.582893 B-MISC word.lower=expovacaciones
4.537645 B-MISC word.lower=diversia
4.478269 B-ORG  word.lower=esquerra
4.460681 B-PER  word.lower=mcmanaman
4.381993 B-ORG  word[-2:]=-e
4.366103 O      word.istitle=False
4.281879 B-LOC  -1:word.lower=nuboso
4.097012 O      postag[:2]=Fp
4.088448 B-ORG  word.lower=terra
4.085507 B-MISC word.lower=diversia.com
4.047451 B-ORG  word.lower=ciu
3.959024 I-LOC  -1:word.lower=calle
3.957171 B-ORG  word[-3:]=CiU

Top negative:
-1.635142 O      -2:word.lower=rodillo
-1.663723 O      word.lower=circo
-1.685685 B-PER  word[-2:]=ón
-1.743277 B-PER  -1:word.lower=en
-1.755077 B-LOC  -2:word.lower=nacional
-1.868276 B-PER  word[-3:]=nes
-1.954365 I-ORG  word[-3:]=La
-1.954822 B

Some observations:

* **8.743642 B-ORG  word.lower=psoe-progresistas** - the model remembered names of some entities - maybe it is overfit, or maybe our features are not adequate, or maybe remembering is indeed helpful;
* **5.195429 I-LOC  -1:word.lower=calle**: "calle" is a street in Spanish; model learns that if a previous word was "calle" then the token is likely a part of location;
* **-3.529449 O      word.isupper=True**, ** -2.913103 O      word.istitle=True **: UPPERCASED or TitleCased words are likely entities of some kind;
* **-2.585756 O      postag=NP** - proper nouns (NP is a proper noun in the Spanish tagset) are often entities.

In [22]:
pickle.dump({'X': X_test, 'y': y_test}, open(CRFSUITE_TEST_DATA_FILE, 'wb'))

NameError: name 'CRFSUITE_TEST_DATA_FILE' is not defined

In [24]:
CRFSUITE_TEST_DATA_FILE = 'conll2002-esp_crfsuite-test-data.dump'
X_test, y_test = pickle.load(open(CRFSUITE_TEST_DATA_FILE, 'rb'))

In [25]:
# -- Load data and crfsuite model and convert them-------------------------

RECREATE = True  # set to True to recreate flexcrf data with new model

CRFSUITE_MODEL_FILE = 'conll2002-esp.crfsuite'
CRFSUITE_TEST_DATA_FILE = 'conll2002-esp_crfsuite-test-data.dump'
FLEXCRF_TEST_DATA_FILE = 'conll2002-esp_flexcrf-test-data.dump'

# crfsuite model
tagger = Tagger()
tagger.open(CRFSUITE_MODEL_FILE)
model = tagger.info()

data = pickle.load(open(CRFSUITE_TEST_DATA_FILE, 'rb'))
print ("test data loaded.")

if RECREATE:
    dataset, thetas = convert_data_to_flexcrf(data, model, n_seq=3)
    pickle.dump({'dataset': dataset, 'thetas': thetas},
                open(FLEXCRF_TEST_DATA_FILE, 'wb'))
else:
    dd = pickle.load(open(FLEXCRF_TEST_DATA_FILE))
    dataset = dd['dataset']
    thetas = dd['thetas']


test data loaded.

converting to flexcrf format...
f_xy_desc created.
t_xyy_desc created
Processing sentence 1/3...
Processing sentence 2/3...
Processing sentence 3/3...


In [26]:

def viterbi_decoder(m_xy, n=None, log_version=True):
    """
    Performs MAP inference, determining $y = \argmax_y P(y|x)$, using the
    Viterbi algorithm.

    Parameters
    ----------
    m_xy : ndarray, shape (n_obs, n_labels, n_labels)
        Values of log-potentials ($\log M_i(y_{i-1}, y_i, x)$)
        computed based on feature functions f_xy and/or user-defined potentials
        `psi_xy`. At t=0, m_xy[0, 0, :] contains values of $\log M_1(y_0, y_1)$
        with $y_0$ the fixed initial state.

    n : integer, default=None
        Time position up to which to decode the optimal sequence; if not
        specified (default) the score is computed for the whole sequence.

    Returns
    -------
    y_pred : ndarray, shape (n_obs,)
        Predicted optimal sequence of labels.

    """
    
    
    n_obs, n_label, _ = m_xy.shape 
    argmax = np.zeros((n_obs - 1, n_label), dtype=int)
    y_pred = np.zeros(n_obs, dtype=int)

    # init
    score = m_xy[0, 0, :]

    # Boucle pour calculer les nouveaux scores 
    for i in range(n_obs - 1):
        tmp = score.reshape(-1, 1) + m_xy[i + 1]
        argmax[i] = np.argmax(tmp, axis=0)
        score = tmp[argmax[i], np.arange(n_label)]

    # On trouve le chemin pour la meilleure séquence 
    y_pred[-1] = np.argmax(score)
    for i in np.arange(n_obs - 1)[::-1]:
        y_pred[i] = argmax[i, y_pred[i + 1]]

    return y_pred, score[y_pred[-1]]


In [27]:
# -- Start classification ------------------------------------------------

for seq in range(len(dataset)):
    # -- with crfsuite
    s_ = tagger.tag(data['X'][seq])
    y_ = np.array([int(model.labels[s]) for s in s_])
    prob_ = tagger.probability(s_)

    print ("\n-- With crfsuite:")
    print ("labels:\n", s_, "\n", y_)
    print ("probability:\t %f" % prob_)

    # -- with flexcrf
    f_xy, y = dataset[seq]
    theta = thetas[seq]

    m_xy, f_m_xy = _compute_all_potentials(f_xy, theta)

    y_pred = viterbi_decoder(m_xy)[0]

    # ADD CODE TO COMPUTE POSTERIOR PROBABILITY WITH FLEXCRF HERE ....

    print ("-- With flexcrf:")
    print ("labels:\n", y_pred)
    print ("equal predictions: ", np.all(y_pred == y_))
    #print "probability:\t %f" % prob
    #print "delta:\t %f" % abs(prob-prob_)

tagger.close()


-- With crfsuite:
labels:
 ['B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O'] 
 [0 7 1 1 1 1 2 1 1]
probability:	 0.586343


C:\Users\Mohamed\MS BGD\MDI341 Machine Learning avancé\TP4 CRF\tp\flexcrf_tp\models\linear_chain.py:218: RuntimeWarning: divide by zero encountered in log
  f_xy.select(y1=ND, y2=y2, arr_out=True).T)
C:\Users\Mohamed\MS BGD\MDI341 Machine Learning avancé\TP4 CRF\tp\flexcrf_tp\models\linear_chain.py:220: RuntimeWarning: divide by zero encountered in log
  f_xy.select(y1=y1, y2=y2, arr_out=True).T)


-- With flexcrf:
labels:
 [0 7 1 1 1 1 2 1 1]
equal predictions:  True

-- With crfsuite:
labels:
 ['O'] 
 [1]
probability:	 0.999946
-- With flexcrf:
labels:
 [1]
equal predictions:  True

-- With crfsuite:
labels:
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O'] 
 [1 1 1 1 1 1 1 1 1 1 1 1 5 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 3 4 4 1 1 1 1 1 0 7 1]
probability:	 0.430171
-- With flexcrf:
labels:
 [1 1 1 1 1 1 1 1 1 1 1 1 5 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 3 4 4 1 1 1 1 1 0 7 1]
equal predictions:  True
